# DMIA sport intro: How frequent is this password?

**Данный ноутбук выполнялся на платформе Kaggle, как Kaggle kernel. Из библиотек машинного обучения использовался только sklearn. В первой версии ноутбука использовался только случайный лес с чуть более глубокими деревьями, чем в данном ноутбуке. Также в нем использовалось разделение на обучающую и валидационную выборки. Увеличение глубины деревьев случайного леса приводило к снижению ошибки, как на обучаюющей, так и на валидационной выборке. Правда это снижение было достаточно скромным. Поэтому для более существенного улучшения результата была обучена модель градиентного бустинга.**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/dmia-sport-2019-fall-intro/Xtest.csv
/kaggle/input/dmia-sport-2019-fall-intro/train.csv
/kaggle/input/dmia-sport-2019-fall-intro/sample_submission.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
train = pd.read_csv('/kaggle/input/dmia-sport-2019-fall-intro/train.csv', dtype={'Times':np.uint16}, keep_default_na=False)
test = pd.read_csv('/kaggle/input/dmia-sport-2019-fall-intro/Xtest.csv', index_col=['Id'], keep_default_na=False)

**В следующих 2-х ячейках - простая функция для определения "среднего расстояния между соседними символами на клавиатуре". Она была написана, чтобы отловить те пароли, в которых идут подряд несколько расположенных рядом (или одинаковых) клавиш. Эта функция считает расстояние между одинаковыми или соседними (в одном ряду) клавишами равным нулю. Впрочем, значение (feature importance) признака, сгенерированного этой функцией, для моделей случайного леса оказалось не очень существенным. А эксперименты с разными вариантами функции показали, что от выбора таких вариантов почти ничего не меняется.**

In [4]:
kbdrows=["`1234567890-=", "~!@#$%^&*()_+", "qwertyuiop[]\\", "asdfghjkl;'", "zxcvbnm,./"]
kbddists = {}
for i1, row1 in enumerate(kbdrows):
    for j1, ch1 in enumerate(row1):
        for i2, row2 in enumerate(kbdrows):
            for j2, ch2 in enumerate(row2):
                if min(i1, i2) == 0 and max(i1, i2) > 1:
                    kbddists[(ch1, ch2)] = abs(i1 - i2 - 1) + abs(j1 - j2)*(abs(j1 - j2) > 1)
                else:
                    kbddists[(ch1, ch2)] = abs(i1 - i2) + abs(j1 - j2)*(abs(j1 - j2) > 1)      

In [5]:
def avedist(s):
    if len(s) < 2:
        return 0
    a = 0;
    for i in range(len(s)-1):
        a += kbddists.get((s[i], s[i+1]), 5)
    return a/(len(s) - 1)

**Далее вычисляются 14 простых признаков: длина пароля, количество уникальных символов в нем, количества и доли уникальных цифр, строчных и прописных букв, всех букв и букв вместе с цифрами среди уникальных символов пароля, а также упомянутое ранее среднее расстояние между соседними символами на клавиатуре. Возможно какие-то из этих признаков извыточны в силу высокой скорелированности, но для случайного леса и градиентного бустинга на деревьях это не помеха.**

In [6]:
df = pd.concat([train.drop('Times', axis=1), test], sort=False)

In [7]:
df['len'] = df.Password.str.len().astype('int8')

In [8]:
digits = set('0123456789')
uppers = set('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
lowers = set('abcdefghijklmnopqrstuvwxyz')

In [9]:
%%time
df['set'] = df.Password.apply(lambda s: set(s))
df['nunique'] = df['set'].apply(len).astype('int8')
df['digit'] = df['set'].apply(lambda x: len(x & digits)).astype('int8')
df['upper'] = df['set'].apply(lambda x: len(x & uppers)).astype('int8')
df['lower'] = df['set'].apply(lambda x: len(x & lowers)).astype('int8')
df['letters'] = df['upper'] + df['lower']
df['alpha'] = df['digit'] + df['letters']
df['nunique_share'] = df['nunique'] / df['len']
df['digit_share'] = df['digit'] / df['nunique']
df['upper_share'] = df['upper'] / df['nunique']
df['lower_share'] = df['lower'] / df['nunique']
df['letters_share'] = df['letters'] / df['nunique']
df['alpha_share'] = df['alpha'] / df['nunique']

CPU times: user 41.4 s, sys: 3.86 s, total: 45.3 s
Wall time: 44.9 s


In [10]:
df.fillna(0, inplace=True)
df.drop('set', axis=1, inplace=True)

In [11]:
%%time
df['avedist'] = df['Password'].str.lower().apply(avedist)

CPU times: user 32.5 s, sys: 368 ms, total: 32.9 s
Wall time: 32.8 s


In [12]:
df.drop('Password', axis=1, inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5189371 entries, 0 to 1037874
Data columns (total 14 columns):
len              int8
nunique          int8
digit            int8
upper            int8
lower            int8
letters          int8
alpha            int8
nunique_share    float64
digit_share      float64
upper_share      float64
lower_share      float64
letters_share    float64
alpha_share      float64
avedist          float64
dtypes: float64(7), int8(7)
memory usage: 351.4 MB


**Далее в качестве признаков добавляются 1000 наиболее часто встречавшихся в паролях обучающей выборки 3-грамм. Ограничившись 3-граммами мы наверняка избежим чрезмерной подстройки под обучающую выборку и, в то же время, получим шанс обучиться на небольшом числе наиболее характерных для "парольного" языка сочитаний символов, которые априори могут быть и не очевидными. (На самом деле, результат работы первой версии этого ноутбука показал, что добавление небольшого числа 4- и 5-грамм чуть-чуть уменьшает значение ошибки на валидационной выборке.)**

In [14]:
%%time
countvect = CountVectorizer(analyzer='char', ngram_range=(3,3), max_features=1000)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 34.8 µs


In [15]:
from scipy.sparse import hstack
sparse_train = hstack([df[:len(train)].values, 
                       countvect.fit_transform(train['Password'])]).tocsr()
y = train['Times']

**Далее обучается случайный лес из 30 деревьев максимальной глубины 11 со случайным выбором порогов для разбиения по каждой переменой (скорее всего, это несколько ускоряет работу алгоритма). На Kaggle этот алгоритм работал целых полчаса.**

In [16]:
%%time 
rf = ExtraTreesRegressor(n_estimators=30, max_depth=11, random_state=11, n_jobs=-1)
rf.fit(sparse_train, np.log1p(y))

CPU times: user 1h 53min, sys: 6.03 s, total: 1h 53min 6s
Wall time: 29min 54s


ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=11,
                    max_features='auto', max_leaf_nodes=None,
                    min_impurity_decrease=0.0, min_impurity_split=None,
                    min_samples_leaf=1, min_samples_split=2,
                    min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=-1,
                    oob_score=False, random_state=11, verbose=0,
                    warm_start=False)

In [17]:
print('Forest msle on train set:', mean_squared_error(rf.predict(sparse_train), np.log1p(y)))

Forest msle on train set: 0.13685412775885444


**С использованием случайного леса среднеквадратическая ошибка на обучающей выборке составила примерно 0.137, а RMSLE - 0.37.**

**Посмотрим теперь на наиболее значимые с точки зрения случайного леса признаки (с большими feature importances).**

In [18]:
f_names = np.r_[df.columns.values, 
                countvect.get_feature_names()]
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
print('Feature ranking')
for f in range(145):
    print('%d. %s (%f)' % (f+1, f_names[indices[f]], importances[indices[f]]))

Feature ranking
1. len (0.208516)
2. digit_share (0.071312)
3. lower_share (0.071040)
4. nunique (0.066824)
5. nunique_share (0.062559)
6. letters_share (0.052412)
7. 198 (0.045000)
8. alpha (0.040708)
9. 197 (0.034777)
10. digit (0.033354)
11. 199 (0.030103)
12. 196 (0.025839)
13. lower (0.019469)
14. letters (0.018818)
15. avedist (0.017446)
16. 119 (0.016680)
17. 219 (0.016009)
18. upper_share (0.011714)
19. 195 (0.008682)
20. 123 (0.008169)
21. 051 (0.007704)
22. upper (0.007591)
23. 081 (0.007001)
24. 011 (0.006604)
25. 041 (0.005826)
26. 121 (0.005801)
27. 071 (0.005451)
28. 061 (0.005160)
29. 111 (0.005100)
30. 021 (0.005000)
31. 031 (0.004581)
32. 200 (0.004260)
33. er1 (0.003182)
34. 091 (0.002684)
35. ter (0.001793)
36. alpha_share (0.001557)
37. 519 (0.001451)
38. 345 (0.001447)
39. 619 (0.001374)
40. 078 (0.001362)
41. 101 (0.001295)
42. 419 (0.001248)
43. 194 (0.001244)
44. 019 (0.001238)
45. 098 (0.001234)
46. ing (0.001219)
47. 234 (0.001105)
48. 193 (0.001091)
49. 719 (

**Первое, что бросается в глаза, помимо созданных ранее признаков заметное значение имеют цифровые 3-граммы 196, 197, 198, 199, которые по-видимому являются началом, используемых в паролях годов рождения.**

**Чтобы понизить ошибку воспользуемся алгоритмом градиентного бустинга. Для ускорения работы используем эксперементальной для sklearn моделью HistGradientBoostingRegressor. К сожалению, данный алгоритм пока не умеет работать с разреженными матрицами. Поэтому для того, чтобы точно не выйти за границы памяти, установленные для Kaggle kernel, мы будем использовать только первые 140 наиболее важных с точки зрения случайного леса признаков.**

In [19]:
del rf

In [20]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
gb = HistGradientBoostingRegressor(max_iter=400, max_leaf_nodes=47, random_state=11)

In [21]:
dense_train = sparse_train[:, indices[:140]].toarray()
del sparse_train
dense_train.shape

(4151496, 140)

In [22]:
%%time
gb.fit(dense_train, np.log1p(y))

CPU times: user 25min 56s, sys: 7.54 s, total: 26min 3s
Wall time: 6min 49s


HistGradientBoostingRegressor(l2_regularization=0.0, learning_rate=0.1,
                              loss='least_squares', max_bins=256,
                              max_depth=None, max_iter=400, max_leaf_nodes=47,
                              min_samples_leaf=20, n_iter_no_change=None,
                              random_state=11, scoring=None, tol=1e-07,
                              validation_fraction=0.1, verbose=0)

In [23]:
print('Boosting msle on train set:', mean_squared_error(gb.predict(dense_train), np.log1p(y)))

Boosting msle on train set: 0.1272702588621421


**Градиентный бустинг считался всего 7 минут, но заметно снизил ошибку в сравнении со случайным лесом. RMSLE на обучающей выборке составляет примерно 0,357. На приватной части тестовой выборки она составила 0.359.**

**Сгенерируем файл ответов для загрузки на Kaggle.**

In [24]:
del dense_train

In [25]:
dense_test = hstack([df[len(train):].values, 
                     countvect.transform(test['Password'])]).tocsr()[:, indices[:140]].toarray()
dense_test.shape

(1037875, 140)

In [26]:
test['Times'] = np.expm1(gb.predict(dense_test))

In [27]:
del dense_test

In [28]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1037875 entries, 0 to 1037874
Data columns (total 2 columns):
Password    1037875 non-null object
Times       1037875 non-null float64
dtypes: float64(1), object(1)
memory usage: 23.8+ MB


In [29]:
test[['Times']].to_csv('submit_2.csv', index_label='Id')

In [30]:
test.sort_values(by='Times')

,Password,Times
Id,,
178387,2081198,0.029114
71214,y0919199,0.108382
318177,1983081,0.127280
387770,j1970091,0.184417
391218,20608153198,0.204502
...,...,...
704375,33333333,348.087630
900990,55555555,348.087630
604320,66666666,348.087630
